In [ ]:
def load_pretrained(load_path):
    with open(load_path, "r") as fin:
        # Optional: depending on the specific format of pretrained vector file
        n, d = map(int, fin.readline().split())
        tokens = []
        embeds = []
        for line in fin:
            line = line.rstrip().split(' ')
            token, embed = line[0], list(map(float, line[1:]))
            tokens.append(token)
            embeds.append(embed)
        vocab = Vocab(tokens)
        embeds = torch.tensor(embeds, dtype=torch.float)
    return vocab, embeds

def knn(W, x, k):
    similarities = torch.matmul(x, W.transpose(1, 0)) / (torch.norm(W, dim=1) * torch.norm(x) + 1e-9)
    knn = similarities.topk(k=k)
    return knn.values.tolist(), knn.indices.tolist()

# 在词向量空间中进行近义词检索(这里我们用到我们之前glove.py训练好的glove.vec，预训练词向量)
def find_similar_words(embeds, vocab, query, k=10):
    knn_values, knn_indices = knn(embeds, embeds[vocab[query]], k + 1)
    knn_words = vocab.convert_ids_to_tokens(knn_indices)
    print(f">>> Query word: {query}")
    for i in range(k):
        print(f"cosine similarity={knn_values[i + 1]:.4f}: {knn_words[i + 1]}")

word_sim_queries = ['good', 'well', 'many', 'fears', 'american', \
         'analyst', 'china', 'apple', 'banana', 'orange', 'grape', 'watermelon']

vocab, embeds = load_pretrained("glove.vec")

for w in word_sim_queries:
    find_similar_words(embeds, vocab, w)